In [1]:
import os 
%pwd

'/home/mrafi/Desktop/Books/Bootcamp/E2EMLOps/18-e2e/MLops_Ds1/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/mrafi/Desktop/Books/Bootcamp/E2EMLOps/18-e2e/MLops_Ds1'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
  root_dir: Path
  STATUS_FILE: Path
  unzip_data_dir: Path
  all_schema: dict

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
  root_dir: Path
  data_path: Path

In [16]:
from src.data_science.constants import CONFIG_FILE_PATH, SCHEMA_FILE_PATH, PARAMS_FILE_PATH
from src.data_science.utils.common import read_yaml, create_dir
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, schema_file_path= SCHEMA_FILE_PATH, params_file_path=PARAMS_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_dir([self.config.artifacts_toot])
        
    def get_dataingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_dir([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
              root_dir= config.root_dir,
              source_URL=config.source_URL,
              local_data_file=config.local_data_file ,
              unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
    def get_datavalidation_config(self)-> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_dir([config.root_dir])
        data_validation_config = DataValidationConfig(
              root_dir= config.root_dir,
              STATUS_FILE=config.STATUS_FILE,
              unzip_data_dir=config.unzip_data_dir ,
              all_schema=schema
        )
        return data_validation_config
    
    def get_datatransformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation
        print(f"config: {config}")
        create_dir([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path=config.data_path,
        )
        return data_transformation_config

In [17]:
from src.data_science import logger
import pandas as pd
from sklearn.model_selection import train_test_split
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path, sep=";")
        train,test = train_test_split(data, test_size=.2)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        
        logger.info("Splitted data into train and test")
        logger.info(f"number of records in training data : {train.shape[0]}")
        logger.info(f"number of records in test data : {test.shape[0]}")
        
        print( f"number of records in training data : {train.shape[0]}")
        print( f"number of records in test data : {test.shape[0]}")
    

In [19]:
try:
    config = ConfigurationManager()
    data_transofrmation_config = config.get_datatransformation_config()
    data_transofrmation = DataTransformation(data_transofrmation_config)
    data_transofrmation.train_test_splitting()
except Exception as e:
    raise e 


[2025-05-03 07:58:47,203: INFO: common: config/config.yaml file loaded sucessfully]
[2025-05-03 07:58:47,205: INFO: common: params.yaml file loaded sucessfully]
[2025-05-03 07:58:47,207: INFO: common: schema.yaml file loaded sucessfully]
[2025-05-03 07:58:47,208: INFO: common: Directory artifacts created]
config: {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}
[2025-05-03 07:58:47,209: INFO: common: Directory artifacts/data_transformation created]
[2025-05-03 07:58:47,227: INFO: 3927080862: Splitted data into train and test]
[2025-05-03 07:58:47,228: INFO: 3927080862: number of records in training data : 1279]
[2025-05-03 07:58:47,229: INFO: 3927080862: number of records in test data : 320]
number of records in training data : 1279
number of records in test data : 320


In [20]:

data_validation_status = config.get_datavalidation_config()


[2025-05-03 08:28:09,747: INFO: common: Directory artifacts/data_validation created]


In [22]:
data_validation_status.STATUS_FILE

'artifacts/data_validation/status.txt'

In [29]:
with open(Path(data_validation_status.STATUS_FILE), "r") as f:
            content = f.read().split()
            print(content)
            if 'True' in content[-1]:
                print("found")

['Validation', 'Sucessful.', 'Validation', 'Status:True']
found
